In [79]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import time
import random
import urllib.parse
from datetime import datetime, timedelta

In [117]:
HEADERS = {
    'accept': '*/*',
    'accept-language': 'en-US',
    'apollographql-client-name': 'nextjs-web',
    'content-type': 'application/json',
    # Update cookie if necessary. Find in gql network tab after loading meetup.com
    'cookie': 'orion_lsid=c11ec6d4-e14a-45a6-9236-23ddf3f47084; SIFT_SESSION_ID=b46fafff-5fa2-4f58-a51e-6afe97e4c0bc; __Host-NEXT_MEETUP_CSRF=74b859d5-aace-4a4b-b4e8-ebbe44c2231b; MEETUP_BROWSER_ID=id=c50f3402-3e13-46b4-8f65-e8921a5debdb; MEETUP_TRACK=id=97030fb6-12ac-4699-9c99-84f188471354; SIFT_SESSION_ID=b46fafff-5fa2-4f58-a51e-6afe97e4c0bc; COOKIE_ID_PICOX_ID=09ce44f9-a3f9-420f-9906-d18afbd88e9d; MEETUP_MEMBER_LOCATION_V2=city=New+York&country=us&localized_country_name=us&state=NY&name_string=New+York%252C+NY%252C+USA&zip=10001&lat=40.75&lon=-73.98999786376953&timeZone=America%252FNew_York; cjConsent=MHxOfDB8Tnww; _hjHasCachedUserAttributes=true; _clck=1dvad71%5E2%5Eg1o%5E0%5E2168; __stripe_mid=ed644de9-6ea3-4df3-9dca-3a87c6e8823aede747; USER_CHANGED_DISTANCE_FILTER=false; _gcl_au=1.1.523011679.1765225801; _gid=GA1.2.1156177322.1765225801; _hjSessionUser_6522278=eyJpZCI6ImNhZGZmMGU5LWY0NTUtNWJiZi1hODZiLTQzMzUyOTQxNDYyOSIsImNyZWF0ZWQiOjE3NjUyMjU3ODMyNDcsImV4aXN0aW5nIjp0cnVlfQ==; _hjSession_6522278=eyJpZCI6IjhkYjUwNDUwLWY2YjctNGJkYS05MzFjLWYwM2MzYmMzMWU5MCIsImMiOjE3NjUyMjg2NTg0MjQsInMiOjAsInIiOjAsInNiIjowLCJzciI6MCwic2UiOjAsImZzIjowLCJzcCI6MH0=; cjUser=60988c8a-5382-4cfc-a664-c510741d4261; __stripe_sid=3e220658-4a7d-42b5-955e-e0d2d66f19536fddd6; OptanonConsent=isGpcEnabled=0&datestamp=Mon+Dec+08+2025+16%3A20%3A22+GMT-0500+(Eastern+Standard+Time)&version=202309.1.0&browserGpcFlag=0&isIABGlobal=false&hosts=&consentId=5ec5251a-bd93-4adc-8e00-416051bc3989&interactionCount=1&landingPath=NotLandingPage&groups=C0002%3A1%2CC0001%3A1%2CC0004%3A1%2CC0003%3A1&AwaitingReconsent=false; _uetsid=a1113770d47411f0aa230dbe0e34335a; _uetvid=a1113b20d47411f09c6fd1942201dc12; _ga=GA1.1.947606036.1765225801; g_state={"i_l":0,"i_ll":1765228823819,"i_b":"EE+AKd99Q9At7gIQEjvnOJJwHzUvJMWpgglSca0UNhA"}; _clsk=p59klj%5E1765229806939%5E8%5E1%5Ea.clarity.ms%2Fcollect; _ga_NP82XMKW0P=GS2.1.s1765228822$o2$g1$t1765229820$j60$l0$h0; MEETUP_MEMBER_LOCATION=city=New+York&country=us&localized_country_name=us&state=NY&name_string=New+York%252C+NY%252C+USA&zip=10001&lat=40.75&lon=-73.98999786376953&timeZone=America%252FNew_York&isCountry=false',
    'origin': 'https://www.meetup.com',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'
}

In [122]:
def get_event_urls(pages_to_scrape=3):
    api_url = "https://www.meetup.com/gql2"
    all_urls = []
    cursor = None # Start from the beginning 
    
    print(f"🚀 Starting API Scrape for {pages_to_scrape} pages...")
    
    for i in range(pages_to_scrape):

    # The payload exactly matches your CURL structure
        payload = {
            "operationName": "recommendedEventsWithSeries",
            "variables": {
                "first": 20,
                "lat": 40.75,
                "lon": -73.98999786376953,
                "startDateRange": "2025-11-27T17:09:47-05:00[US/Eastern]",
                "numberOfEventsForSeries": 5,
                "seriesStartDate": "2025-11-27",
                "sortField": "RELEVANCE",
                "doConsolidateEvents": True,
                "doPromotePaypalEvents": False,
                "indexAlias": "{\"filterOutWrongLanguage\": \"true\",\"modelVersion\": \"split_offline_online\"}",
                "dataConfiguration": "{\"isSimplifiedSearchEnabled\": true, \"include_events_from_user_chapters\": true}",
                "after": cursor # The magic pagination key
            },
            "extensions": {
                "persistedQuery": {
                    "version": 1,
                    "sha256Hash": "4eda170f69bd7288f7433435dedbd1b2192b7351f8e5bc7b067e27a51d4974d2"
                }
            }
        }

        try:
            response = requests.post(api_url, headers=HEADERS, json=payload)
            data = response.json()
            # Navigate the JSON structure you provided: data -> result -> edges
            result_data = data.get('data', {}).get('result', {})
            edges = result_data.get('edges', [])
            if not edges:
                print("❌ No more results found.")
                break
        

            # Extract URLs from this batch
            batch_urls = []
            for edge in edges:
                node = edge.get('node', {})
                url = node.get('eventUrl')
                if url:
                    batch_urls.append(url)
                    
            all_urls.extend(batch_urls)
            print(f" Page {i+1}: Found {len(batch_urls)} events. (Cursor: {cursor})")
            # Update Cursor for next loop
            page_info = result_data.get('pageInfo', {})
            if page_info.get('hasNextPage'):
                cursor = page_info.get('endCursor')
            else:
                print(" Reached the end of the list.")
                break
            time.sleep(random.uniform(1, 3)) # Polite wait
        except Exception as e:
            print(f" ⚠️ Error on page {i+1}: {e}")
            break
    
    return list(set(all_urls)) # Remove duplicates

In [129]:
def get_event_urls(pages_to_scrape=3):
    api_url = "https://www.meetup.com/gql2"
    all_urls = []
    cursor = None  # Start from the beginning

    print(f"🚀 Starting API Scrape for {pages_to_scrape} pages...")

    for i in range(pages_to_scrape):
        # The payload exactly matches your CURL structure
        payload = {
            "operationName": "recommendedEventsWithSeries",
            "variables": {
                "first": 20,
                "lat": 40.75,
                "lon": -73.98999786376953,
                "startDateRange": "2025-11-27T17:09:47-05:00[US/Eastern]",
                "numberOfEventsForSeries": 5,
                "seriesStartDate": "2025-11-27",
                "sortField": "RELEVANCE",
                "doConsolidateEvents": True,
                "doPromotePaypalEvents": False,
                "indexAlias": "{\"filterOutWrongLanguage\": \"true\",\"modelVersion\": \"split_offline_online\"}",
                "dataConfiguration": "{\"isSimplifiedSearchEnabled\": true, \"include_events_from_user_chapters\": true}",
                "after": cursor  # The magic pagination key
            },
            "extensions": {
                "persistedQuery": {
                    "version": 1,
                    "sha256Hash": "cf6348a7edb376af58158519e78130eb8beced0aaaed60ab379e82f25fd52eea"
                }
            }
        }

        try:
            response = requests.post(api_url, headers=HEADERS, json=payload)
            data = response.json()

            # Navigate the JSON structure you provided: data -> result -> edges
            result_data = data.get('data', {}).get('result', {})
            edges = result_data.get('edges', [])

            if not edges:
                print("❌ No more results found.")
                break

            # Extract URLs from this batch
            batch_urls = []
            for edge in edges:
                node = edge.get('node', {})
                url = node.get('eventUrl')
                if url:
                    batch_urls.append(url)

            all_urls.extend(batch_urls)
            print(f"✅ Page {i+1}: Found {len(batch_urls)} events. (Cursor: {cursor})")

            # Update Cursor for next loop
            page_info = result_data.get('pageInfo', {})
            if page_info.get('hasNextPage'):
                cursor = page_info.get('endCursor')
            else:
                print("✅ Reached the end of the list.")
                break

            time.sleep(random.uniform(1, 3))  # Polite wait

        except Exception as e:
            print(f"⚠️ Error on page {i+1}: {e}")
            break

    return list(set(all_urls))  # Remove duplicates


In [43]:
def get_event_details(url):
    try:
        response = requests.get(url, headers=HEADERS)
        if response.status_code != 200: return None

        soup = BeautifulSoup(response.text, 'html.parser')
        script_tag = soup.find('script', id='__NEXT_DATA__')
        if not script_tag: return None
        
        data = json.loads(script_tag.string)
        
        # Access the massive database of all objects on the page
        apollo_state = data.get('props', {}).get('pageProps', {}).get('__APOLLO_STATE__', {})
        
        # 1. Find the Main Event Object
        # Try standard props first
        event_data = data.get('props', {}).get('pageProps', {}).get('event')
        
        # If missing, search Apollo State manually
        if not event_data:
            for key, value in apollo_state.items():
                if key.startswith("Event:") and value.get('eventUrl') == url:
                    event_data = value
                    break
        
        if not event_data: return None

        # --- EXTRACT FIELDS ---
        title = event_data.get('title')
        date_time = event_data.get('dateTime')
        
        # Resolve Venue (Using helper in case it's a ref)
        venue_raw = event_data.get('venue')
        venue = resolve_apollo_data(venue_raw, apollo_state) or {}
        
        location = "Online"
        if venue.get('name'):
            location = f"{venue.get('name')}, {venue.get('city', 'NYC')}"

        # --- ROBUST TOPIC EXTRACTION ---
        categories = []
        
        # A. Try Event Topics
        raw_topics = event_data.get('topics')
        resolved_topics = resolve_apollo_data(raw_topics, apollo_state)
        
        # B. Fallback to Group Topics
        if not resolved_topics:
            group_raw = event_data.get('group')
            group = resolve_apollo_data(group_raw, apollo_state) or {}
            raw_group_topics = group.get('topics')
            resolved_topics = resolve_apollo_data(raw_group_topics, apollo_state)

        # C. Parse the Resolved List
        if isinstance(resolved_topics, list):
            for t in resolved_topics:
                if isinstance(t, dict) and 'name' in t:
                    categories.append(t['name'])
                elif isinstance(t, str):
                    categories.append(t)

        return {
            "Title": title,
            "DateTime": date_time,
            "Location": location,
            "Categories": categories, 
            "URL": url
        }

    except Exception as e:
        print(f"Skipping {url}: {e}")
        return None

In [130]:
urls = get_event_urls(pages_to_scrape=30)

🚀 Starting API Scrape for 30 pages...
✅ Page 1: Found 18 events. (Cursor: None)
✅ Page 1: Found 18 events. (Cursor: None)
✅ Page 2: Found 17 events. (Cursor: MjA=)
✅ Page 2: Found 17 events. (Cursor: MjA=)
✅ Page 3: Found 19 events. (Cursor: NDA=)
✅ Page 3: Found 19 events. (Cursor: NDA=)
✅ Page 4: Found 20 events. (Cursor: NjA=)
✅ Page 4: Found 20 events. (Cursor: NjA=)
✅ Page 5: Found 19 events. (Cursor: ODA=)
✅ Page 5: Found 19 events. (Cursor: ODA=)
✅ Page 6: Found 18 events. (Cursor: MTAw)
✅ Page 6: Found 18 events. (Cursor: MTAw)
✅ Page 7: Found 18 events. (Cursor: MTIw)
✅ Page 7: Found 18 events. (Cursor: MTIw)
✅ Page 8: Found 19 events. (Cursor: MTQw)
✅ Page 8: Found 19 events. (Cursor: MTQw)
✅ Page 9: Found 18 events. (Cursor: MTYw)
✅ Page 9: Found 18 events. (Cursor: MTYw)
✅ Page 10: Found 17 events. (Cursor: MTgw)
✅ Page 10: Found 17 events. (Cursor: MTgw)
✅ Page 11: Found 19 events. (Cursor: MjAw)
✅ Page 11: Found 19 events. (Cursor: MjAw)
✅ Page 12: Found 19 events. (Curso

In [131]:
details_list = []
for idx, url in enumerate(urls):
    print(f"🔍 Scraping event {idx+1}/{len(urls)}: {url}")
    details = get_event_details(url)
    if details:
        details_list.append(details)

🔍 Scraping event 1/483: https://www.meetup.com/nyc_socials/events/312097081/
🔍 Scraping event 2/483: https://www.meetup.com/opendiner/events/310932264/
🔍 Scraping event 3/483: https://www.meetup.com/ai-terraform-meetup/events/311951888/
🔍 Scraping event 4/483: https://www.meetup.com/spirituality-social-meetup-of-nyc/events/312253398/
🔍 Scraping event 5/483: https://www.meetup.com/new-york-coworking-friends/events/312174579/
🔍 Scraping event 6/483: https://www.meetup.com/new-york-city-uws-kadima-meetup-group/events/312320370/
🔍 Scraping event 7/483: https://www.meetup.com/downtown-jane-austen-meetup-for-austenites/events/312253741/
🔍 Scraping event 4/483: https://www.meetup.com/spirituality-social-meetup-of-nyc/events/312253398/
🔍 Scraping event 5/483: https://www.meetup.com/new-york-coworking-friends/events/312174579/
🔍 Scraping event 6/483: https://www.meetup.com/new-york-city-uws-kadima-meetup-group/events/312320370/
🔍 Scraping event 7/483: https://www.meetup.com/downtown-jane-austen

KeyboardInterrupt: 

In [32]:
print(details_list[1])

{'Title': 'Meet New Friends Where Connection Begins and Loneliness Ends', 'DateTime': '2025-12-18T18:30:00-05:00', 'Location': 'Online event, ', 'Categories': ['Friendship for the Lonely', 'Make New Friends', 'Self-Empowerment', 'Self-Help & Self-Improvement', 'Loneliness'], 'URL': 'https://www.meetup.com/opendiner/events/310932264/'}


In [34]:
# 1. Create DataFrame
df = pd.DataFrame(details_list)

# 2. Convert DateTime to object
df['DateTime'] = pd.to_datetime(df['DateTime'], utc=True)

# 3. Preview
print("Dataframe Shape:", df.shape)
display(df.head(10))

Dataframe Shape: (492, 5)


,Title,DateTime,Location,Categories,URL
0,Myers Briggs Personality Type Mingle & Afterpa...,2025-12-13 20:00:00+00:00,"115 E 57th Street, NY","[Dining Out, Conversation, Fun Times, Social, ...",https://www.meetup.com/nyc_socials/events/3120...
1,Meet New Friends Where Connection Begins and L...,2025-12-18 23:30:00+00:00,"Online event,","[Friendship for the Lonely, Make New Friends, ...",https://www.meetup.com/opendiner/events/310932...
2,Lunch & Learn: Building Trust in AI-Generated ...,2025-12-10 17:00:00+00:00,"1460 Broadway, New York","[Artificial Intelligence, Cloud Security, Infr...",https://www.meetup.com/ai-terraform-meetup/eve...
3,Spiritual Social Get-Together @ IBM Atrium,2025-12-14 19:00:00+00:00,"IBM Atrium, New York","[Community, Philosophy, Spirituality, Social, ...",https://www.meetup.com/spirituality-social-mee...
4,Coworking Session & Job Search/Salary Negotiat...,2025-12-15 15:00:00+00:00,"Serendipity Labs, New York","[Career Coaching, Coworking, Make New Friends,...",https://www.meetup.com/new-york-coworking-frie...
5,Join for Ping Pong in midtown (UWS Kadima group),2025-12-14 20:00:00+00:00,"Spin midtown, New York","[Make New Friends, Socializing, Ping Pong, Exe...",https://www.meetup.com/new-york-city-uws-kadim...
6,Happy Birthday Jane!,2025-12-17 00:00:00+00:00,"The Liberty, New York","[Book Club, Books and Drinks, Literature, Jane...",https://www.meetup.com/downtown-jane-austen-me...
7,Focal Point - LIC,2025-12-12 00:00:00+00:00,"Focal Point Beer Co, Queens","[European Board Games, Tabletop Role Playing a...",https://www.meetup.com/lic-board-games/events/...
8,Bringing Down The House,2025-12-21 00:30:00+00:00,"Online event,","[Stand-up Comedy, Locals & New in Town, Make N...",https://www.meetup.com/we-watch-films-shows-an...
9,NYC Airflow Meetup at Astronomer HQ!,2025-12-17 22:30:00+00:00,"Astronomer HQ, New York City","[Big Data, Data Analytics, Data Science, Data ...",https://www.meetup.com/nyc-apache-airflow-meet...


In [ ]:
df_w_urls = df.copy()

if pd.api.types.is_string_dtype(df_w_urls['Categories']):
    df_w_urls['Categories'] = df_w_urls['Categories'].str.split(', ')

df_w_urls['DateTime'] = pd.to_datetime(df_w_urls['DateTime'], utc=True)

df_w_urls.to_json('event_data/meetup_events_urls.json', orient='records', indent=4, force_ascii=False)

print(f"✅ Successfully saved {len(df_w_urls)} events to 'event_data/meetup_events_urls.json'")

✅ Successfully saved 492 events to 'meetup_events_urls.json'


In [ ]:
df_no_urls = df[['Title', 'DateTime', 'Location', 'Categories']].copy()

if pd.api.types.is_string_dtype(df_no_urls['Categories']):
    df_no_urls['Categories'] = df_no_urls['Categories'].str.split(', ')

df_no_urls['DateTime'] = pd.to_datetime(df_no_urls['DateTime'], utc=True)

df_no_urls.to_json('event_data/meetup_events.json', orient='records', indent=4, force_ascii=False)

print(f"✅ Successfully saved {len(df_no_urls)} events to 'event_data/meetup_events.json'")

✅ Successfully saved 492 events to 'meetup_events.json'


In [133]:
import requests
import json
import time
import random
import urllib.parse
from datetime import datetime, timedelta

# ---------------------------------------------------------
# 1. SETUP
# ---------------------------------------------------------
# !!! CRITICAL: Paste a NEW Cookie from your browser !!!
HEADERS = {
    'authority': 'www.meetup.com',
    'accept': '*/*',
    'content-type': 'application/json',
    'origin': 'https://www.meetup.com',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36',
    'cookie': 'orion_lsid=...' 
}

# ---------------------------------------------------------
# 2. HELPERS
# ---------------------------------------------------------
def get_meetup_timestamp(dt_obj, is_end=False):
    # Fixed: Removed [US/Eastern] suffix to match browser payload
    # Format: 2026-01-08T00:00:00-05:00
    time_part = "23:59:59" if is_end else "00:00:00"
    date_str = dt_obj.strftime('%Y-%m-%d')
    return f"{date_str}T{time_part}-05:00"

def get_referer_url(start_str, end_str):
    # Generates the exact URL found in the browser to use as Referer
    s_enc = urllib.parse.quote(start_str)
    e_enc = urllib.parse.quote(end_str)
    return f"https://www.meetup.com/find/?location=us--ny--new-york&source=EVENTS&customStartDate={s_enc}&customEndDate={e_enc}"

# ---------------------------------------------------------
# 3. MAIN SCRAPER
# ---------------------------------------------------------
def get_event_urls(months_to_scrape=6):
    api_url = "https://www.meetup.com/gql2"
    all_urls = []
    
    # Anchor Date (Today) stays static
    series_start_date_fixed = datetime.now().strftime('%Y-%m-%d')
    current_search_date = datetime.now()

    # --- MANUAL STRINGS (To match browser whitespace exactly) ---
    # Python adds spaces after commas by default. Meetup rejects that in the Hash check.
    # We hardcode this string to ensure it is byte-perfect.
    index_alias_str = "\"{\\\"filterOutWrongLanguage\\\": \\\"true\\\",\\\"modelVersion\\\": \\\"split_offline_online\\\"}\""
    
    # This one uses standard spacing, so we can hardcode it too for safety
    data_config_str = "{\"isSimplifiedSearchEnabled\": true, \"include_events_from_user_chapters\": true}"

    print(f"🚀 Starting Scrape. Anchor: {series_start_date_fixed}")

    for i in range(months_to_scrape):
        # Window: 30 days
        window_start = current_search_date + timedelta(days=(i * 30))
        window_end = window_start + timedelta(days=29)
        
        start_str = get_meetup_timestamp(window_start)
        end_str = get_meetup_timestamp(window_end, is_end=True)
        
        print(f"\n📅 Month {i+1}: {start_str[:10]} -> {end_str[:10]}")

        # FIX: Update Referer for every month
        curr_headers = HEADERS.copy()
        curr_headers['referer'] = get_referer_url(start_str, end_str)

        cursor = None
        has_next = True
        
        while has_next:
            variables = {
                "first": 20,
                "lat": 40.75,
                "lon": -73.98999786376953,
                "startDateRange": start_str,
                "endDateRange": end_str,
                "numberOfEventsForSeries": 5,
                "seriesStartDate": series_start_date_fixed,
                "sortField": "RELEVANCE",
                "doConsolidateEvents": True,
                "doPromotePaypalEvents": False,
                "indexAlias": index_alias_str,
                "dataConfiguration": data_config_str
            }
            
            # FIX: Only add 'after' key if cursor exists (Matches browser behavior)
            if cursor:
                variables['after'] = cursor

            payload = {
                "operationName": "recommendedEventsWithSeries",
                "variables": variables,
                "extensions": {
                    "persistedQuery": {
                        "version": 1,
                        "sha256Hash": "cf6348a7edb376af58158519e78130eb8beced0aaaed60ab379e82f25fd52eea"
                    }
                }
            }

            try:
                response = requests.post(api_url, headers=curr_headers, json=payload)
                
                if response.status_code != 200:
                    print(f"   ❌ Status: {response.status_code} (Check Cookie)")
                    has_next = False
                    break

                data = response.json()
                
                if 'errors' in data:
                    print(f"   ❌ API Error: {data['errors'][0]['message']}")
                    has_next = False
                    break

                result_data = data.get('data', {}).get('recommendedEventsWithSeries', {})
                edges = result_data.get('edges', [])
                
                if not edges:
                    print("   ⚠️ No events returned for this window.")
                    has_next = False
                    break
                
                count = 0
                for edge in edges:
                    url = edge.get('node', {}).get('eventUrl')
                    if url:
                        all_urls.append(url)
                        count += 1
                
                print(f"   Collected {count} events...")

                page_info = result_data.get('pageInfo', {})
                has_next = page_info.get('hasNextPage', False)
                cursor = page_info.get('endCursor')
                
                time.sleep(random.uniform(1, 2))

            except Exception as e:
                print(f"   ❌ Error: {e}")
                has_next = False
        
        time.sleep(2)

    return list(set(all_urls))

# Run
final_links = get_event_urls(months_to_scrape=6)
print(f"\n🎉 Total Unique Links: {len(final_links)}")

🚀 Starting Scrape. Anchor: 2025-12-08

📅 Month 1: 2025-12-08 -> 2026-01-06
   ⚠️ No events returned for this window.

📅 Month 2: 2026-01-07 -> 2026-02-05
   ⚠️ No events returned for this window.

📅 Month 3: 2026-02-06 -> 2026-03-07
   ⚠️ No events returned for this window.


KeyboardInterrupt: 